In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from scipy import sparse
from scipy import optimize
from scipy import signal
from functools import reduce
from collections import defaultdict
import pprint
from tqdm.notebook import tqdm
import pandas as pd # Only here to print arrays nicely

In [2]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

# TOC
1. [1x1 Beam Functions](#1x1)
2. [NxN Beam Complications](#NxN)
3. [Playground](#playground)

### Simplified case for 1x1 beam <a name="1x1"></a>

In [3]:
def get_num_baselines(Nside):
    # For square of N telescopes, should be 2N-2*sqrt(N) + 1
    # Fix to a corner and there are N-1 options for unique baselines. 
    # Flip over to other side (over x or y axis) and get another N-1 options
    # Duplicated are the pure x-axis and pure y-axis so -2*(sqrt(N)-1)
    # Final +1 is from 0 baseline
    
    N_bases = 2*Nside**2 - 2*Nside
    return int(N_bases)

rand_phases = lambda x: np.random.uniform(0, 2*np.pi, x)
zero_weight = lambda x, d: x/d if d else 0

def make_gains(Nside):
    # Create complex gains with either (amplitude, phase) or (real, imaginary)
    Nant = Nside**2
    gain_amp = np.random.normal(1, .05, Nant)
    gain_phase = rand_phases(Nant)
    tgain = gain_amp*np.exp(1j*gain_phase)    
    return tgain

def make_data(Nside, gains, noise=0.1):
    Nant = Nside**2
    Nbase = get_num_baselines(Nside)
    vis_true = np.random.normal(0,1,size=(Nbase,2)).view(np.complex128).flatten() ## size of unique baselines
    ant_i, ant_j, visndx, data = [], [], [], []
    ndx=0
    ndx2base={}
    base2ndx={}
    for i in range(Nant):
        xi,yi=np.unravel_index(i,(Nside,Nside))
        for j in range (i+1,Nant):
            xj,yj=np.unravel_index(j,(Nside,Nside))
            assert (xj>=xi)
            baseline = (xj-xi,yj-yi)
            if baseline in base2ndx:
                cndx = base2ndx[baseline]
            else:
                cndx = ndx
                base2ndx[baseline]=ndx
                ndx2base[ndx]=baseline
                ndx+=1
            ant_i.append(i)
            ant_j.append(j)
            visndx.append(cndx)
            data.append(vis_true[cndx]*gains[i]*np.conj(gains[j]))
            
    assert(ndx==Nbase)
    ant_i = np.array(ant_i)
    ant_j = np.array(ant_j)
    visndx = np.array(visndx)
    data = np.array(data)
    noise = np.random.normal(0,noise,size=(len(data),2)).view(np.complex128).flatten() ## size of unique baselines
    data += noise
    return vis_true, data, ant_i, ant_j, visndx, ndx2base, base2ndx

In [4]:
def make_pred(gains, vis, ant_i, ant_j, visndx):
    gains_i = gains[ant_i]
    cgains_j = np.conj(gains[ant_j])
    pred = gains_i*cgains_j*vis[visndx]
    return pred

In [5]:
def chi2(data, gains, vis, ant_i, ant_j, visndx, noise=0.1):
    pred = make_pred(gains, vis, ant_i, ant_j, visndx)
    chi2 = np.abs((data - pred)**2).sum()/(noise**2)
    dof = len(data)*2
    return chi2, dof

### NxN beam complications <a name="NxN"></a>

In [6]:
def get_weighted_array(alpha, Nspacing, numdraws=1e5):
    Nbeam = Nspacing**2
    rmax = alpha*.5
    ndraws = int(numdraws)
    spacing = np.linspace(0,1,Nspacing+1)
    centered_spacing = spacing - .5
    empty_weight_beam = np.zeros((Nspacing, Nspacing))
    
    for i in range(Nbeam):
        xi, yi = np.unravel_index(i, (Nspacing, Nspacing))
        draws = np.array([np.random.uniform(centered_spacing[xi], centered_spacing[xi+1], ndraws), np.random.uniform(centered_spacing[yi], centered_spacing[yi+1], ndraws)])
        dist = np.linalg.norm(draws, axis=0)
        empty_weight_beam[xi, yi] = np.sum(dist < rmax)/ndraws
    sym_beam = .5*(empty_weight_beam + empty_weight_beam.T)
    return sym_beam

In [319]:
def n_mesh(br):
    nmesh = lambda x: tuple(np.meshgrid(range(x[0]-br, x[0]+br+1), range(x[1]-br, x[1]+br+1), indexing='ij'))
    return nmesh

five_mesh = n_mesh(2)

In [320]:
def make_uv_grid(Nside):
    uv_size = Nside*2 - 1
    center = (Nside-1,Nside-1)
    npcenter = np.array(center)
    img_size = (uv_size, uv_size)
    random_image = np.random.normal(0, 1, img_size)
    centered_uv = np.fft.fftshift(np.fft.fft2(random_image))
#     centered_uv[center] = 0
    topleft_uv = np.fft.ifftshift(centered_uv)
    
    return centered_uv, topleft_uv, npcenter

In [321]:
def make_visibilities(Nside, beam_radius):
    n = Nside
    br = beam_radius
    
    # Useful shapes/sizes to have on hand
    orig_shape = (2*n-1, n)
    new_shape = tuple([i*br + (br - 1) for i in orig_shape])
    new_size = np.prod(new_shape)
    center = (int((new_shape[0]-1)/2), br-1)
    
    #Matrices of visibiltiies and possible indices
    random_vis = np.random.normal(0, 1, (new_shape[0], new_shape[1], 2)).view(np.complex128)
    poss_index = np.arange(new_size).reshape(new_shape)
    
    #Clear off the leftmost columns to remove redundant conjugate stuff
    poss_index[0:center[0], 0:br] = 0
    poss_index[center[0]:, 0:br-1] = 0
    
    oversampled_baselines = poss_index.nonzero()[0].shape[0]
    visib = np.zeros(oversampled_baselines, dtype=np.complex128)
    #Baseline - index dictionaries
    new_ndx = 0
    n2b = {}
    b2n = {}
    n2true = {}
    true2n = {}
    for i in range(new_size):
        xi,yi = np.unravel_index(i, new_shape)
        if poss_index[xi,yi] == 0:
            continue
        else:
            baseline = (xi-center[0], yi-center[1])
            if baseline[0] < 0:
                baseline = (-1*baseline[0], -1*baseline[1])

            if baseline in b2n:
                cndx = new_ndx
            else:
                cndx = new_ndx
                b2n[baseline] = new_ndx
                n2b[new_ndx] = baseline
                if tuple(np.mod(baseline, br)) == (0,0):
                    modtuple = tuple(np.floor_divide(baseline, 3))
                    true2n[modtuple] = new_ndx
                    n2true[new_ndx] = modtuple
                new_ndx += 1
        visib[cndx] = random_vis[xi,yi]
    return n2b, b2n, n2true, true2n, visib

In [352]:
def vis_to_grid(visib, ndx2base, size):
    # Double the size to make sure to avoid rewriting values with negative indexing
    uv_size = (2*size[0], 2*size[1])
    new_grid = np.zeros(uv_size, dtype=np.complex128)
    for i,v in enumerate(visib):
        base = ndx2base[i]
        inv_base = tuple(-1*np.array(base))
        new_grid[base] = v
        new_grid[inv_base] = np.conj(v)
    return new_grid

In [353]:
def make_data_grid(Nside, gains, conv_beam, noise=0.1, verbose=True):
    Nant = Nside**2
    Nbase = get_num_baselines(Nside)
    beam_radius = int((conv_beam.shape[0]-1)/2) + 1
    mesh = n_mesh(beam_radius-1)
    
    n2b, b2n, n2t, t2n, visib = make_visibilities(Nside, beam_radius)
    if verbose:
        print("Made visib")
    orig_shape = (2*Nside-1, Nside)
    new_shape = tuple([i*beam_radius + (beam_radius - 1) for i in orig_shape])
    
    vis_grid = vis_to_grid(visib, n2b, new_shape)
    if verbose:
        print("Made grid")
    
    ant_i, ant_j, visndx, data, datandx = [], [], [], [], []
    
    for i in range(Nant):
        xi,yi=np.unravel_index(i,(Nside, Nside))
        for j in range (i+1,Nant):
            xj,yj=np.unravel_index(j,(Nside, Nside))
            assert (xj>=xi)
            baseline = (xj-xi,yj-yi)
            
            virtual_n = t2n[baseline]
            grid_base = n2b[virtual_n]
            
            virtual_points = mesh(grid_base)
            data_sum = (vis_grid[virtual_points].T * gains[i] * np.conj(gains[j]) * conv_beam).sum()
            
            ant_i.append(i)
            ant_j.append(j)
            visndx.append(virtual_n)
            data.append(data_sum)
            datandx.append(virtual_points)
    
    if verbose:
        print("Created data")
    
    ant_i = np.array(ant_i)
    ant_j = np.array(ant_j)
    visndx = np.array(visndx)
    data = np.array(data)
    noise = np.random.normal(0,noise,size=(len(data),2)).view(np.complex128).flatten() ## size of unique baselines
    data += noise
    return visib, data, ant_i, ant_j, visndx, datandx, n2b, b2n, n2t, t2n, vis_grid

In [354]:
def vector_b2n(b2n, datandx):
    newndx = []
    for l in datandx:
        x,y = l
        flattened = []
        ndx_size = x.size
        ndx_shape = x.shape
        for i in range(ndx_size):
            point = np.unravel_index(i, ndx_shape)
            k = np.array([x[point], y[point]])
            key = tuple(k)
            if key in b2n:
                flattened.append(b2n[key])
            else:
                key = tuple(-1*k)
                flattened.append(-1*b2n[key])
        newndx.append(flattened)
    newndx = np.array(newndx)
    return newndx

In [355]:
def conjugate_visib(vis, ndxs):
    flat = []
    for i in ndxs:
        if i >= 0:
            flat.append(vis[i])
        else:
            flat.append(np.conj(vis[-1*i]))
    return np.array(flat)

In [356]:
def forward_model(vis, gains, conv_beam, ant_i, ant_j, n2b, datandx):
    Nside = int(np.sqrt(len(gains_true)))
    beam_radius = int((conv_beam.shape[0]-1)/2) + 1
    
    orig_shape = (2*Nside-1, Nside)
    new_shape = tuple([i*beam_radius + (beam_radius - 1) for i in orig_shape])
    vis_grid = vis_to_grid(vis, n2b, new_shape)
    
    pred = []
    
    for i in range(len(ant_i)):
        pred.append((vis_grid[datandx[i]] * gains[ant_i[i]] * np.conj(gains[ant_j[i]]) * conv_beam).sum())
    pred = np.array(pred)
    return pred

In [357]:
Nside = 15
Nant = Nside * Nside
Nbase = get_num_baselines(Nside)
# gains_true = make_gains(Nside)
gains_true = np.ones(Nant, dtype=np.complex128)
# uv_grid, npcenter = make_uv_grid(Nside)
# vis_true, data, ant_i, ant_j, visndx, ndx2base, base2ndx = make_data(Nside, gains_true)
weighted_beam = get_weighted_array(1, 3)
conv_beam = signal.convolve(weighted_beam, weighted_beam)
vis_true, data, ant_i, ant_j, visndx, datandx, n2b, b2n, n2t, t2n, tl_grid = make_data_grid(Nside, gains_true, conv_beam, noise=0.1)
flatndx = vector_b2n(b2n, datandx)
data_len = len(data)
no_noise = make_pred(gains_true, vis_true, ant_i, ant_j, visndx)

Made visib
Made grid
Created data


In [328]:
np.sum(tl_grid[datandx[0]].T*conv_beam)

(-21.758139569614144+15.395245031421835j)

In [329]:
test_b = forward_model(vis_true, gains_true, conv_beam, ant_i, ant_j, n2b, datandx)

In [330]:
test_b[0]

(-21.758139569614144+15.395245031421835j)

In [33]:
# print(np.sum(signal.convolve(weighted_beam, weighted_beam)))
# print(np.sum(weighted_beam))
# print(np.sum(uv_grid[data_ndx[0]]))
print(data[0]/np.sum(tl_grid[datandx[0]]))
# print(data[0]/np.sum(weighted_beam))

(1.0054034714561515-0.002289114396490778j)


In [188]:
sol, n2r, _, score, A, b = solve_grid(data, data_ndx, np.sum(weighted_beam), weighted_beam.flatten())
finsize = np.product(uv_grid.shape)
real_sol = np.array([uv_grid[np.unravel_index(n2r[i], uv_grid.shape)] for i in np.arange(len(n2r))])

In [189]:
print("Real sol: ", np.sum(np.abs(A@real_sol - b)))
print("lin sol: ", np.sum(np.abs(A@sol - b)))

Real sol:  13436.846278151952
lin sol:  3643.9062704599846


In [187]:
phased_gains = gains_true*np.exp(1j*rand_phases(Nant)/5)
chi2(data, gains_true, vis_true, ant_i, ant_j, visndx)

(3124100516.744245, 600)

### Below this is my random testing/playground <a name="playground"></a>

In [403]:
real_part = data.real
imag_part = data.imag

In [404]:
v_size = len(set(np.abs(flatndx).flatten())) + 1

In [405]:
realA = np.zeros((data_len, v_size))

In [406]:
for i,v in enumerate(flatndx):
    absv = np.abs(v)
    realA[i][absv] = conv_beam.flatten()

In [408]:
shitsol = np.linalg.lstsq(realA, real_part)

<ipython-input-408-1e2be06cd092>:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  shitsol = np.linalg.lstsq(realA, real_part)


KeyboardInterrupt: 

In [387]:
b2n[(0,0)]

1936